In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.DataStructs import cDataStructs
import re
import random
import numpy as np
import pandas as pd
from random import seed
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt

In [3]:
prot_df = pd.read_csv('../PDB_proteins_90pc_identity_6lu7.csv', engine='python', delimiter=',')

In [5]:
prot_df.columns

Index(['pdb_id', 'organism_scientific_name', 'tax_id', 'organism_synonyms',
       'rank', 'genus', 'superkingdom', 'molecule_name', 'molecule_synonym',
       'molecule_type', 'entity_id', 'modified_residue_flag', 'mutation_type',
       'uniprot_accession', 'uniprot_id', 'gene_name', 'interacting_molecules',
       'interacting_ligands', 'compound_id', 'compound_name',
       'compound_systematic_name', 'entity_weight', 'number_of_polymers',
       'number_of_polymer_residues', 'number_of_polymer_entities',
       'number_of_bound_entities', 'number_of_protein_chains',
       'max_observed_residues'],
      dtype='object')

In [8]:
#Compound ID's. Not informative
prot_df[["interacting_ligands","compound_id"]]

,compound_id,interacting_ligands
0,NaN,NaN
1,MPD,"MPD : (4S)-2-METHYL-2\,4-PENTANEDIOL"
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,ATO,ATO : CHLOROACETONE
6,I12,I12 : N-[(5-METHYLISOXAZOL-3-YL)CARBONYL]-L-AL...
7,NaN,NaN
8,NaN,NaN
9,"EDO,CL","CL : CHLORIDE ION,EDO : 1\,2-ETHANEDIOL"
